In [1]:
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import numpy as np
import datetime
import os

from importlib import reload

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only use the first GPU
  try:
    tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
  except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
    print(e)

1 Physical GPUs, 1 Logical GPU


In [3]:
DR_LEVELS_PER_CLASS = [[0], [1,2,3,4]]

IMAGE_SIZE = (540, 540, 3)

# Specify dataset files
TRAIN_FILE = 'DATASET-VALIDATION-10.csv'
VALIDATION_FILE = 'DATASET-VALIDATION-10.csv'

BATCH_SIZE = 32

In [4]:
def read_image(path, label):
    return tf.image.decode_png(tf.io.read_file(path)), label

def normalize(img, label):
    return tf.cast(img, tf.float32) / 255., label

In [5]:
train_csv = pd.read_csv(TRAIN_FILE)

asignations = [-1 for i in DR_LEVELS_PER_CLASS for j in i] # [[0], [1], [2,3,4]] --> [-1, -1, -1, -1, -1]
for i, class_ in enumerate(DR_LEVELS_PER_CLASS):
    for dr_lvl in class_:
        asignations[dr_lvl] = i
            
dr_lvls = [asignations[i] for i in train_csv['DR_level'].tolist()]

# print(asignations)
# print(dr_lvls[:20])

dataset_train = tf.data.Dataset.from_tensor_slices((train_csv['path'].tolist(), dr_lvls))
# print(train_csv['DR_level'].tolist()[:20])

In [6]:
dataset_train = dataset_train.map(read_image)

In [7]:
dataset_train = dataset_train.map(normalize)

In [8]:
dataset_train = dataset_train.batch(BATCH_SIZE)

In [9]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32,(5,5), strides=2, input_shape=IMAGE_SIZE),
    tf.keras.layers.ReLU(), # 270, 270
    tf.keras.layers.Conv2D(32,(7,7), strides=5),
    tf.keras.layers.ReLU(), # 54, 54
    tf.keras.layers.Conv2D(32,(3,3), strides=2),
    tf.keras.layers.ReLU(), # 27, 27
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(48, activation='relu'),
#     tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Dense(len(DR_LEVELS_PER_CLASS), activation='softmax')
])

# COMO LA SALIDA ES DADA POR NUMEROS ENTEROS, SE UTILIZA SPARSE_CATEGORICAL_CROSSENTROPY
# ------------
# Use this crossentropy loss function when there are two or more label classes. We expect labels to be provided as integers. 
# If you want to provide labels using one-hot representation, please use CategoricalCrossentropy loss. There should be # classes 
# floating point values per feature for y_pred and a single floating point value per feature for y_true.
# ------------
# https://www.tensorflow.org/api_docs/python/tf/keras/losses/SparseCategoricalCrossentropy

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])

In [10]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

cbacks = [tf.keras.callbacks.TensorBoard('logs/Modelo_basico_EYEPACS_Seq', histogram_freq=1, write_graph=False)]

In [14]:
# See Tensorboard
%tensorboard --logdir logs

Reusing TensorBoard on port 6006 (pid 2790), started 23:10:54 ago. (Use '!kill 2790' to kill it.)

In [12]:
model.fit(dataset_train, epochs=50, validation_data=dataset_train, verbose=1, callbacks=cbacks)

Epoch 1/50
  1/222 [..............................] - ETA: 0s - loss: 0.7567 - accuracy: 0.2812WARNING:tensorflow:From /home/alumno/miguel_herrera/lib/python3.8/site-packages/tensorflow/python/ops/summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
222/222 [==============================] - 94s 426ms/step - loss: 0.5749 - accuracy: 0.7445 - val_loss: 0.5730 - val_accuracy: 0.7467
Epoch 2/50
222/222 [==============================] - 63s 283ms/step - loss: 0.5634 - accuracy: 0.7463 - val_loss: 0.5656 - val_accuracy: 0.7467
Epoch 3/50
222/222 [==============================] - 64s 287ms/step - loss: 0.5601 - accuracy: 0.7466 - val_loss: 0.5657 - val_accuracy: 0.7467
Epoch 4/50
222/222 [==============================] - 62s 281ms/step - loss: 0.5583 - accuracy: 0.7467 - val_loss: 0.5534 - val_accuracy: 0.7464
Epoch 5/50
222/222 [=====================

# Modelo ajustado (usando 'one-hot')

In [6]:
dataset_train_2 = tf.data.Dataset.from_tensor_slices((train_csv['path'].tolist(), tf.keras.utils.to_categorical(dr_lvls, num_classes=len(DR_LEVELS_PER_CLASS))))

dataset_train_2 = dataset_train_2.map(read_image)
dataset_train_2 = dataset_train_2.map(normalize)
dataset_train_2 = dataset_train_2.batch(BATCH_SIZE)

In [11]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

cbacks = [tf.keras.callbacks.TensorBoard('logs/EYEPACS_Seq_Modelo_ajustado', histogram_freq=1, write_graph=False)]

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [12]:
model_2 = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32,(5,5), strides=2, input_shape=IMAGE_SIZE),
    tf.keras.layers.ReLU(), # 270, 270
    tf.keras.layers.Conv2D(32,(7,7), strides=5),
    tf.keras.layers.ReLU(), # 54, 54
    tf.keras.layers.Conv2D(32,(3,3), strides=2),
    tf.keras.layers.ReLU(), # 27, 27
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(48, activation='relu'),
#     tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Dense(len(DR_LEVELS_PER_CLASS), activation='softmax')
])

model_2.compile(optimizer='adam',
                loss='categorical_crossentropy', 
                metrics=['accuracy'])

In [13]:
model_2.fit(dataset_train_2, epochs=50, validation_data=dataset_train_2, verbose=1, callbacks=cbacks)

Epoch 1/50
  1/222 [..............................] - ETA: 0s - loss: 0.6665 - accuracy: 0.7188WARNING:tensorflow:From /home/alumno/miguel_herrera/lib/python3.8/site-packages/tensorflow/python/ops/summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
222/222 [==============================] - 73s 328ms/step - loss: 0.5760 - accuracy: 0.7467 - val_loss: 0.5745 - val_accuracy: 0.7467
Epoch 2/50
222/222 [==============================] - 73s 329ms/step - loss: 0.5631 - accuracy: 0.7467 - val_loss: 0.5609 - val_accuracy: 0.7467
Epoch 3/50
222/222 [==============================] - 74s 334ms/step - loss: 0.5631 - accuracy: 0.7460 - val_loss: 0.5571 - val_accuracy: 0.7467
Epoch 4/50
222/222 [==============================] - 77s 348ms/step - loss: 0.5581 - accuracy: 0.7467 - val_loss: 0.5594 - val_accuracy: 0.7467
Epoch 5/50
222/222 [=====================